In [3]:
# import all necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import git

In [4]:
### Define file paths
base_root = git.Repo('.', search_parent_directories=True).working_tree_dir
base_input = base_root + '\\Input & Output\\Input\\'
base_output = base_root + '\\Input & Output\\Output\\'

weather_input = base_input + 'Weather\\'
weather_output = base_output + 'Weather\\'

In [5]:
### Combine all weather data into one dataframe
df_weather = pd.read_csv(weather_input + 'weatherstats_toronto_daily.csv')
df_hdd_cdd = pd.read_csv(weather_input + 'CYYZ Heating Cooling Combined.csv')
df_weather = df_weather.rename(columns = {
    'date': 'Date',
    'heatdegdays': 'HDD 18',
    'cooldegdays': 'CDD 18'
})

df_all_weather = df_weather.merge(df_hdd_cdd, how = 'left', on = 'Date')
df_all_weather = df_all_weather.iloc[::-1]

C:\Users\CGOD\AppData\Local\Temp\ipykernel_47364\2881632096.py:2: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv(weather_input + 'weatherstats_toronto_daily.csv')


In [6]:
### Process the combined dataframe
df_all_weather['Date'] = pd.to_datetime(df_all_weather['Date'])
group_lst = ['avg_hourly_temperature', 'avg_hourly_relative_humidity', 'avg_hourly_dew_point', 'avg_hourly_wind_speed', 'avg_hourly_pressure_sea', 'daylight', 'HDD 15.5', 'CDD 15.5', 'HDD 18', 'CDD 18']

# monthly
df_all_weather_monthly_avg = df_all_weather.groupby(pd.Grouper(key='Date', freq='ME'))[group_lst].mean().reset_index()
df_all_weather_monthly_avg['Date'] = df_all_weather_monthly_avg['Date'].dt.strftime('%Y-%m')
df_precipitation_monthly_sum = df_all_weather.groupby(pd.Grouper(key='Date', freq='ME'))['precipitation'].sum().reset_index()
df_precipitation_monthly_sum['Date'] = df_precipitation_monthly_sum['Date'].dt.strftime('%Y-%m')

# yearly
df_all_weather_yearly_avg = df_all_weather.groupby(pd.Grouper(key='Date', freq='YE'))[group_lst].mean().reset_index()
df_all_weather_yearly_avg['Date'] = df_all_weather_yearly_avg['Date'].dt.strftime('%Y')
df_precipitation_yearly_sum = df_all_weather.groupby(pd.Grouper(key='Date', freq='YE'))['precipitation'].sum().reset_index()
df_precipitation_yearly_sum['Date'] = df_precipitation_yearly_sum['Date'].dt.strftime('%Y')

# combine the sum and mean columns
df_all_weather_monthly = df_all_weather_monthly_avg.merge(df_precipitation_monthly_sum, how = 'left', on = 'Date')
df_all_weather_yearly = df_all_weather_yearly_avg.merge(df_precipitation_yearly_sum, how = 'left', on = 'Date')

# reoder the df
order_lst = ['Date'] + group_lst + ['precipitation']
df_all_weather = df_all_weather[order_lst]

In [7]:
### Outputting dataframes
with pd.ExcelWriter(weather_output + 'weather_time_series_data.xlsx') as writer:
    df_all_weather.to_excel(writer, sheet_name='daily average', index=False)

    df_all_weather_monthly.to_excel(writer, sheet_name='monthly average', index=False)
    df_all_weather_yearly.to_excel(writer, sheet_name='yearly average', index=False)